# Chain-of-Thought Prompting

Breaking the instructions down into steps for the model so that it can better understand how to perform the instructions and overall task. 

In [8]:
# Warning control
import warnings
warnings.filterwarnings("ignore")

In [9]:
# Importing libraries
import openai
import os
from dotenv import load_dotenv

# Loading environment variables
load_dotenv()

from openai import OpenAI
client = OpenAI()

In [10]:
def get_chat_completion(messages, model="gpt-4o", temperature=0.0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    return response.choices[0].message.content

## Chain-of-Thought Prompting

**The idea of chain-of-thought (CoT) prompting** -- CoT break down the steps and analyses you want the model to perform so it aids the model in understanding how to achieve the task better. The step-by-step nature of CoT prompts help the model better follow the instructions.

**Where is CoT useful?** -- It's useful in technical domains (science, maths, coding) and where extensive analysis is required to perform a task (market research, report analysis, complex customer support workflows).

In [4]:
menu_delimiter = "<menu_items>"
menu_delimiter_end = "</menu_items>"

thinking_delimiter = "<thinking>"
thinking_delimiter_end = "</thinking>"

response_delimiter = "<response>"
response_delimiter_end = "</response>"

food_items = """
Menu: Kids Menu    
Food Item: Mini Cheeseburger
Price: $6.99
Vegan: N
Popularity: 4/5
Included: Mini beef patty, cheese, lettuce, tomato, and fries.

Menu: Appetizers
Food Item: Loaded Potato Skins
Price: $8.99
Vegan: N
Popularity: 3/5
Included: Crispy potato skins filled with cheese, bacon bits, and served with sour cream.

Menu: Appetizers
Food Item: Bruschetta
Price: $7.99
Vegan: Y
Popularity: 4/5
Included: Toasted baguette slices topped with fresh tomatoes, basil, garlic, and balsamic glaze.

Menu: Main Menu
Food Item: Grilled Chicken Caesar Salad
Price: $12.99
Vegan: N
Popularity: 4/5
Included: Grilled chicken breast, romaine lettuce, Parmesan cheese, croutons, and Caesar dressing.

Menu: Main Menu
Food Item: Classic Cheese Pizza
Price: $10.99
Vegan: N
Popularity: 5/5
Included: Thin-crust pizza topped with tomato sauce, mozzarella cheese, and fresh basil.

Menu: Main Menu
Food Item: Spaghetti Bolognese
Price: $14.99
Vegan: N
Popularity: 4/5
Included: Pasta tossed in a savory meat sauce made with ground beef, tomatoes, onions, and herbs.

Menu: Vegan Options
Food Item: Veggie Wrap
Price: $9.99
Vegan: Y
Popularity: 3/5
Included: Grilled vegetables, hummus, mixed greens, and a wrap served with a side of sweet potato fries.

Menu: Vegan Options
Food Item: Vegan Beyond Burger
Price: $11.99
Vegan: Y
Popularity: 4/5
Included: Plant-based patty, vegan cheese, lettuce, tomato, onion, and a choice of regular or sweet potato fries.

Menu: Desserts
Food Item: Chocolate Lava Cake
Price: $6.99
Vegan: N
Popularity: 5/5
Included: Warm chocolate cake with a gooey molten center, served with vanilla ice cream.

Menu: Desserts
Food Item: Fresh Berry Parfait
Price: $5.99
Vegan: Y
Popularity: 4/5
"""

system_message = f"""
Your task is answer questions factually about a food menu, provided below and delimited by {menu_delimiter}{menu_delimiter_end}.

Step 1: The first step is to check if the user is asking a question related to any type of food (even if that food item is not on the menu). If the question is about any type of food, we move on to Step 2 and ignore the rest of Step 1. If the question is not about food, then we send a response: "Sorry! I cannot help with that. Please let me know if you have a question about our food menu."

Step 2: In this step, we check that the user question is relevant to any of the items on the food menu. You should check that the food item exists in our menu first. If it doesn't exist then send a kind response to the user that the item doesn't exist in our menu and then include a list of available but similar food items without any other details (e.g., price). The food items available are provided below and delimited by {menu_delimiter}{menu_delimiter_end}:

{menu_delimiter}
{food_items}
{menu_delimiter_end}

Step 3: If the item exist in our food menu and the user is requesting for specific information, provide that relevant information to the user using the food menu. Make sure to use a friendly tone and keep the response concise.

Perform the following reasoning steps to send a response to the user:
{thinking_delimiter}
Step 1: <Step 1 reasoning>
Step 2: <Step 2 reasoning>
{thinking_delimiter_end}
{response_delimiter}
Response to the user: <response to user>
{response_delimiter_end}
"""

In [11]:
user_message = "What is the price of the Mini Cheeseburger?"

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message}
]

response = get_chat_completion(messages)
print(response)

<thinking>
Step 1: The user is asking a question related to a food item, specifically the Mini Cheeseburger.
Step 2: The Mini Cheeseburger exists in our menu under the Kids Menu category.
</thinking>
<response>
Response to the user: The price of the Mini Cheeseburger is $6.99. Enjoy your meal!
</response>


## Inner Monologue

You can separate out the reasoning from the final user response.

In [12]:
# extract the final response only
try:
    final_response = response.split(response_delimiter)[1].split(response_delimiter_end)[0]
    final_response = final_response.split("Response to the user: ")[1]
    print(final_response)
except:
    print("No final response found")

The price of the Mini Cheeseburger is $6.99. Enjoy your meal!



## Testing CoT

Let's test a few user queries for analyzing the consistency of the CoT reasoning steps. This is important when experimenting with CoT prompts to analyze more closely where the model is doing okay and where it might need more work. 

In [7]:
user_questions = ["Do you have a kids' menu?", "Do you have any vegan options?", "How much for the shoes?", "Do you have mac & cheese?", "How much for the BBQ?", "What's the price for the mac & cheese?", "What's your most popular dish?"]

for question in user_questions:
    user_message = question
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
    response = get_chat_completion(messages)
    print("Question:", question)
    print("Response:", response)
    print("====================================================")
    

Question: Do you have a kids' menu?
Response: <thinking>
Step 1: The user is asking a question related to food, specifically about a kids' menu.
Step 2: The question is relevant to the food menu provided. The menu includes a "Kids Menu" section.
</thinking>
<response>
Response to the user: Yes, we do have a kids' menu! It includes a Mini Cheeseburger.
</response>
Question: Do you have any vegan options?
Response: <thinking>
Step 1: The user is asking a question related to food, specifically vegan options.
Step 2: The question is relevant to the items on the food menu. We have vegan options available.
</thinking>
<response>
Response to the user: Yes, we have vegan options available! You can choose from the Bruschetta, Veggie Wrap, Vegan Beyond Burger, and Fresh Berry Parfait.
</response>
Question: How much for the shoes?
Response: <thinking>
Step 1: The user is asking about shoes, which is not related to any type of food.
</thinking>
<response>
Response to the user: Sorry! I cannot help

### A word on Native CoT?

More recent models -- also referred to as large reasoning model (LRMs) -- have been trained extensively with reinforcement learning (RL) to natively support chain-of-thought step-by-step reasoning and generated as part of the model response. We are covering this in the upcoming Reasoning chapter of this course.

Hand-written (manual) CoT prompting is still useful for when using non-reasoning models like GPT-4o for tasks that don't require extensive reasoning and when you need to optimize for latency and cost. We will compare and discuss in more detail -- in the upcoming Reasoning chapter -- when to use manual CoT, native CoT, and a combination of both.

### Things to try

- Similar to what we did in few-shot prompting, develop a small testing set (i.e, develop a few questions with ideal chatbot answers) to assess the performance of your CoT prompt
- Create different variants of your CoT prompt and log the results which you can then use to compare and systematically check on improvements
- CoT improvement to try: 
    - Try to separate the thinking steps so you can chain the results instead (this will be covered in more detail in an upcoming lesson)
    - Combine CoT with a few demonstrations on the kinds of answers or step-by-step breakdown or outputs you expect from the model.